In [1]:
!pip install -q geemap earthengine-api pandas numpy


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os, time, json
import ee
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

from IPython.display import Image
from IPython.display import IFrame

In [3]:
ee.Authenticate()

True

In [4]:
ee.Initialize(project='ee-marina-kilifi')

In [5]:
os.makedirs("frontend_data", exist_ok=True)

In [6]:
url = 'https://ee-marina-kilifi.projects.earthengine.app/view/kilifi-ndvi'
IFrame(url, width=1300, height=600)

In [16]:
kilifi_geometry = ee.Geometry.Polygon([
    [39.75, -3.15],
    [40.15, -3.15],
    [40.15, -3.95],
    [39.75, -3.95],
    [39.75, -3.15]
])

In [17]:
def ndvi_median_for_year(year: int) -> ee.Image:
    year = int(year)
    start = f"{year}-01-01"
    end   = f"{year}-12-31"

    if year <= 2012:
        # Landsat 7 TOA
        col = (ee.ImageCollection('LANDSAT/LE07/C02/T1_TOA')
               .filterBounds(kilifi_geometry)
               .filterDate(start, end))
        col = col.map(lambda img: img.normalizedDifference(['B4','B3']).rename('NDVI'))
    else:
        # Landsat 8 SR
        col = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
               .filterBounds(kilifi_geometry)
               .filterDate(start, end))

        def add_ndvi_l8sr(img):
            nir = img.select('SR_B5').multiply(0.0000275).add(-0.2)
            red = img.select('SR_B4').multiply(0.0000275).add(-0.2)
            ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
            return ndvi

        col = col.map(add_ndvi_l8sr)

    # If empty, return a constant NDVI=-0.1 instead of crashing
    return ee.Algorithms.If(
        col.size().gt(0),
        col.median().clip(kilifi_geometry),
        ee.Image.constant(-0.1).rename('NDVI').clip(kilifi_geometry)
    )

In [18]:
years = [1999, 2004, 2009, 2014, 2019, 2020, 2021, 2022, 2023]

def mean_ndvi(img: ee.Image) -> ee.Number:
    return img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=kilifi_geometry,
        scale=30,
        maxPixels=1e13
    ).get('NDVI')

mean_vals = []
for y in years:
    print("Processing year:", y)
    img = ee.Image(ndvi_median_for_year(y))
    val = ee.Number(mean_ndvi(img)).getInfo()
    mean_vals.append(val)

ndvi_df = pd.DataFrame({'year': years, 'mean_ndvi': mean_vals})

# Save
import os, json
os.makedirs("frontend_data", exist_ok=True)
ndvi_df.to_csv("frontend_data/kilifi_ndvi_summary.csv", index=False)
with open("frontend_data/kilifi_ndvi_summary.json", "w") as f:
    json.dump(ndvi_df.to_dict(orient="records"), f, indent=2)

ndvi_df


Processing year: 1999
Processing year: 2004
Processing year: 2009
Processing year: 2014
Processing year: 2019
Processing year: 2020
Processing year: 2021
Processing year: 2022
Processing year: 2023


,year,mean_ndvi
0,1999,0.130197
1,2004,0.080513
2,2009,0.078533
3,2014,0.267574
4,2019,0.240280
5,2020,0.352359
6,2021,0.262297
7,2022,0.223798
8,2023,0.313109


In [19]:
# Global Mangrove Watch v3
extent_1996 = ee.FeatureCollection(
    "projects/earthengine-legacy/assets/projects/sat-io/open-datasets/GMW/extent/gmw_v3_1996_vec"
).filterBounds(kilifi_geometry)

extent_2020 = ee.FeatureCollection(
    "projects/earthengine-legacy/assets/projects/sat-io/open-datasets/GMW/extent/gmw_v3_2020_vec"
).filterBounds(kilifi_geometry)

# Export to local GeoJSONs (geemap handles the EE export+download)
geemap.ee_export_vector(extent_1996, filename='frontend_data/kilifi_mangroves_1996.geojson')
geemap.ee_export_vector(extent_2020, filename='frontend_data/kilifi_mangroves_2020.geojson')

print("Wrote:",
      "frontend_data/kilifi_mangroves_1996.geojson",
      "frontend_data/kilifi_mangroves_2020.geojson", sep="\n- ")

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Wesley Kuria\OneDrive\Desktop\marina\greenrootsproject\scripts\frontend_data\kilifi_mangroves_1996.geojson
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Wesley Kuria\OneDrive\Desktop\marina\greenrootsproject\scripts\frontend_data\kilifi_mangroves_2020.geojson
Wrote:
- frontend_data/kilifi_mangroves_1996.geojson
- frontend_data/kilifi_mangroves_2020.geojson


In [20]:
# Optional: run this if you want GeoTIFFs in your Drive (for GIS/archives)
def export_ndvi_to_drive(image: ee.Image, description: str, folder: str):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder,
        fileNamePrefix=description,
        region=kilifi_geometry,
        scale=30,
        maxPixels=1e13
    )
    task.start()
    print(f"Started export: {description}")
    # Poll until done
    while task.active():
        print("  ...still running")
        time.sleep(20)
    print(f"Done: {description} (state: {task.status().get('state')})")

# Example: export a few years
# for y in [2019, 2020, 2023]:
#     export_ndvi_to_drive(ndvi_median_for_year(y), f'kilifi_ndvi_{y}', 'NDVI_Images')

In [21]:
m = geemap.Map()
m.centerObject(kilifi_geometry, 10)
m.addLayer(ndvi_median_for_year(2023), {'min': -0.2, 'max': 0.8, 'palette': ['blue', 'white', 'green']}, 'NDVI 2023')
m.addLayer(extent_1996, {}, 'Mangroves 1996', False)
m.addLayer(extent_2020, {}, 'Mangroves 2020', False)
m

Map(center=[-3.549963821895521, 39.94999999999978], controls=(WidgetControl(options=['position', 'transparent_…